In [1]:
from agent.agent_functions import FileChange, command_planner

In [2]:
change = FileChange(name='backend/agent/agent.py', changes="Let's update this file using Pydantic Types.")

In [3]:
change.save()

/Users/josephblazick/Documents/GPT-CodeApp/backend/agent/agent.py
# import os
import openai
import json
from typing import List, Optional

# from agent.agent_function


In [3]:
plan = command_planner(
'''
The project directory is currently setup as follows:
+--GPT-CodeApp
    +--README.md
    +--frontend
        +--tailwind.config.js
        +--next.config.js
        +--README.md
        +--postcss.config.js
        +--styles
            +--globals.css
        +--components
            +--modal_bar_modals
            +--SearchBar.js
            +--ModelSelector.js
            +--ModalBar.js
            +--ChatInput.js
            +--RightSidebar.js
            +--EditFilesModal.js
            +--ChatBox.js
            +--LeftSidebar.js
        +--store
            +--sidebar
                +--sidebarSlice.js
            +--index.js
            +--messages
                +--messagesSlice.js
        +--pages
            +--index.js
    +--backend
        +--database
            +--UpdateHandler.py
            +--my_codebase.py
        +--agent
            +--openai_function_call.py
            +--memory_manager.py
            +--agent.py
            +--agent_functions.py
        +--.pytest_cache
            +--README.md
        +--tests
            +--.pytest_cache
                +--README.md
            +--test_codebase.py
            +--test_memory_manager.py
        +--main.py

File: frontend/components/ModalBar.js
Content:
import React, { useState, useEffect } from 'react';
import { get_encoding } from "@dqbd/tiktoken";
// import CustomModal from './CustomModal';
import ReactModal from 'react-modal';

ReactModal.setAppElement('#__next');

const encoding = get_encoding("cl100k_base");

const ModalBar = () => {
    const [systemPrompt, setSystemPrompt] = useState("");
    const [systemTokens, setSystemTokens] = useState(0);
    const [isModalOpen, setIsModalOpen] = useState(false);
    const [editablePrompt, setEditablePrompt] = useState("");
    const [functions, setFunctions] = useState([]);
    const [functionTokens, setFunctionTokens] = useState(0);
    const [isFunctionModalOpen, setIsFunctionModalOpen] = useState(false);
    const [messageHistory, setMessageHistory] = useState([]);
    const [isMessageModalOpen, setIsMessageModalOpen] = useState(false);
    const [messageTokens, setMessageTokens] = useState(0);

    const fetchSystemPromptAndOpenModal = (modal = true) => {
        fetch('http://127.0.0.1:8000/system_prompt')
            .then(response => response.json())
            .then(data => {
                setEditablePrompt(data.system_prompt);
                setSystemPrompt(data.system_prompt);
                const tokens = encoding.encode(data.system_prompt);
                setSystemTokens(tokens.length);
                if (modal) {
                    setIsModalOpen(true);
                }
            })
            .catch(console.error);
    };

    const fetchFunctionsAndOpenModal = (modal = true) => {
        fetch('http://127.0.0.1:8000/get_functions')
            .then(response => response.json())
            .then(data => {
                setFunctions(data.functions);
                if (modal) {
                    setIsFunctionModalOpen(true);
                }
                const f_tokens = encoding.encode(JSON.stringify(data.functions[0]));
                setFunctionTokens(f_tokens.length);
            })
            .catch(console.error);
    };

    const fetchMessagesAndOpenModal = (modal = true) => {
        setMessageHistory([]);
        fetch('http://127.0.0.1:8000/get_messages')
            .then(response => response.json())
            .then(data => {
                setMessageHistory(data.messages);
                if (modal) {
                    setIsMessageModalOpen(true);
                }
                let m_tokens = 0;
                for (const message of data.messages) {
                    m_tokens += encoding.encode(message.content).length;
                }
                console.log(m_tokens);
                setMessageTokens(m_tokens);
            })
            .catch(console.error);
    };

    const updateSystemPrompt = (e) => {
        e.preventDefault();
        fetch('http://127.0.0.1:8000/update_system', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ system_prompt: editablePrompt }),
        })
            .then(response => {
                if (!response.ok) throw new Error('Network response was not ok');
                setSystemPrompt(editablePrompt); // Update original systemPrompt
                setIsModalOpen(false); // Close modal
            })
            .catch(console.error);
    };

    useEffect(() => {
        fetchSystemPromptAndOpenModal(false);
        fetchFunctionsAndOpenModal(false);
        fetchMessagesAndOpenModal(false);
    }, []); //
    return (
        <div className='flex flex-row mx-auto pb-3'>
            <div>
                <button onClick={fetchSystemPromptAndOpenModal}>
                    System Prompt:
                    <span className="ml-2 inline-block bg-green-500 text-white text-xs px-2 rounded-full uppercase font-semibold tracking-wide">
                        {systemTokens} tokens
                    </span>
                </button>
                <ReactModal
                    isOpen={isModalOpen}
                    onRequestClose={() => setIsModalOpen(false)}
                    shouldCloseOnOverlayClick={true}
                    className="fixed inset-0 flex items-center justify-center m-96"
                    overlayClassName="fixed inset-0 bg-black bg-opacity-50"
                >
                    <div className="relative bg-white rounded p-4 w-full max-w-screen-lg mx-auto text-gray-900">
                        <h2 className="text-xl">System Prompt</h2>
                        <p className="text-sm text-green-800">Tokens: {systemTokens}</p>
                        <form onSubmit={updateSystemPrompt}>
                            <textarea
                                value={editablePrompt}
                                onChange={(e) => setEditablePrompt(e.target.value)}
                                className="mt-2 w-full h-96 p-2 border border-gray-300 rounded"
                            />
                            <button
                                type="submit"
                                className="mt-4 px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600"
                            >
                                Update System Prompt
                            </button>
                        </form>
                    </div>
                </ReactModal>
            </div>
            <div className='px-5'>
                < button onClick={fetchFunctionsAndOpenModal} >
                    Functions:
                    <span className="ml-2 inline-block bg-green-500 text-white text-xs px-2 rounded-full uppercase font-semibold tracking-wide">
                        {functionTokens} tokens
                    </span>
                </button >

                <ReactModal
                    isOpen={isFunctionModalOpen}
                    onRequestClose={() => setIsFunctionModalOpen(false)}
                    shouldCloseOnOverlayClick={true}
                    className="fixed inset-0 flex items-center justify-center m-96 w-auto"
                    overlayClassName="fixed inset-0 bg-black bg-opacity-50"
                >
                    <div className="relative bg-white rounded p-4 max-w-screen-lg mx-auto text-gray-900 overflow-scroll">
                        <h2 className="text-xl">Functions</h2>
                        <pre>
                            {functions?.map((f) => (
                                <div key={f?.name}>
                                    <h3 className="text-lg">{f?.name}</h3>
                                    <p>{f?.description}</p>
                                    <hr />
                                </div>
                            ))}
                        </pre>
                    </div>
                </ReactModal>

            </div >
            <div className='px-5'>
                <button onClick={fetchMessagesAndOpenModal}>
                    Message History:
                    <span className="ml-2 inline-block bg-green-500 text-white text-xs px-2 rounded-full uppercase font-semibold tracking-wide">
                        {messageTokens} tokens
                    </span>
                </button>
                <ReactModal
                    isOpen={isMessageModalOpen}
                    onRequestClose={() => setIsMessageModalOpen(false)}
                    shouldCloseOnOverlayClick={true}
                    className="fixed inset-0 flex items-center justify-center m-96"
                    overlayClassName="fixed inset-0 bg-black bg-opacity-50"
                >
                    <div className="bg-gray-700 rounded p-4 max-w-screen-lg max-h-96 overflow-y-scroll mx-auto text-gray-200">
                        <h2 className="text-xl text-white">Messages</h2>
                        {messageHistory.map((m, index) => (
                            <div key={index} className={m.role === 'user' ? 'my-4 p-2 rounded bg-blue-600 text-white' : 'my-4 p-2 rounded bg-green-600 text-white'}>
                                <h3 className="text-lg font-bold">{m.role.toUpperCase()}</h3>
                                <pre className="text-md whitespace-pre-wrap">{m.content}</pre>
                            </div>
                        ))}
                    </div>
                </ReactModal>
            </div>
        </div >
    );
}

export default ModalBar;

INSTRUCTIONS:
In the modal_bar_modals directory (create if necessary), create the following files:
    SystemPromptModal.js: This file will contain the code for the system prompt modal.
    FunctionsModal.js: This file will contain the code for the functions modal.
    MessageHistoryModal.js: This file will contain the code for the message history modal.
Move modal code: Extract the code for each modal from ModalBar.js and move it into the corresponding new file. Make sure to maintain the same functionality and state management in each new component.

Refactor ModalBar.js: In ModalBar.js, refactor the code to import and use the new modal components. Replace the inline modal code with component tags, like <SystemPromptModal />, <FunctionsModal />, and <MessageHistoryModal />.

Export new components: Make sure to export each new component from its file using export default ComponentName.

Test the refactored code: After refactoring, thoroughly test the application to ensure that the modals still work as expected. Check that the state is managed correctly and that all user interactions produce the expected results.

Use a new git branch with a random name. Commit and push changes: Once you've verified that everything is working correctly, commit your changes and push them to the repository. Include a descriptive commit message outlining the changes you've made.

Please let me know if you need any further clarification or have any other questions.
'''
)

ValidationError: 3 validation errors for CommandPlan
command_graph.1.new_file.additional_relevant_files
  Field required [type=missing, input_value={'name': 'frontend/compon...e system prompt modal.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.3/v/missing
command_graph.2.new_file.additional_relevant_files
  Field required [type=missing, input_value={'name': 'frontend/compon...r the functions modal.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.3/v/missing
command_graph.3.new_file.additional_relevant_files
  Field required [type=missing, input_value={'name': 'frontend/compon...message history modal.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.3/v/missing

In [5]:
plan

CommandPlan(command_graph=[Command(id=1, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/SystemPromptModal.js', description='This file will contain the code for the system prompt modal.')), Command(id=2, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/FunctionsModal.js', description='This file will contain the code for the functions modal.')), Command(id=3, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/MessageHistoryModal.js', description='This file will contain the code for the message history modal.')), Command(id=4, command_type=<CommandType.FILE_CHANGE: 'file_change'>, dependent_commands=[1, 2, 3], command_line=None, file_c

In [ ]:
plan.execute()